# Projecting Carson Wentz's Trades impact on the Philadelphia Eagles 2022 Draft Picks

By Kai Canoll

### Synopsis:
The trade between the Eagles and the Colts of Carson Wentz for a 2020 3rd round pick and a conditional 2021 2nd round pick was much discussed. It is certainly one of the weirder trades out there in terms of it's conditions. In short, in order for the Eagles to recieve a 1st round pick, Wentz has to either play:

    1) 75% of offensive snaps for the Colts.
    2) 70% of offensives snaps and the Colts have to make the playoffs. 
    
If these conditions are not met, the Eagles will instead recieve a 2nd round pick from the Colts.
    
Now, as you can imagine, this leaves a huge distribution of possible draft picks the Eagles could recieve from the 1st and 2nd round. 

Each final compensatory pick has it's own probability of occurence, however, and I am here to try an project it!

### Sources:
Look here for how I got my info:

Sportsipy has good stats info for free on players:
https://sportsreference.readthedocs.io/en/stable/examples.html

538 has good projections on the season:
https://data.fivethirtyeight.com/#nfl-elo

Fantasy Pros has info on Carsons Rolling Offensive Snap % Count by Week
https://www.fantasypros.com/nfl/reports/snap-counts/qb.php?year=2021&show=perc&print=true

## Change Variables of Interest Here

In [1]:
Sims=10000 #Number of Simulations, more the longer

Carson_Injury_Risk=.113 #This is the big question, on a per game basis

## Function Definition

In [2]:

#Creates barplot at the from the dataframe with and without sliders
def Plotly_Bar_Plot(Fact1,Value,df,text_hover="",slider_text="",Color_Titler=""
                    ,titler=""
                    ,category_order=False,Sort_fact="",Preset_Order=[]
                    ,category_order_array={},vmin="",vmax="",xmin="",xmax="",barmoder='relative',cmin="",cmax=""):
    if type(vmin) == str:
        vmin = min(df[Value])
    if type(vmax) == str:
        vmax=  max(df[Value])
    if type(xmin) == str:
        xmin = min(df[Fact1])
    if type(xmax) == str:
        xmax=  max(df[Fact1])
        

    yaxiser=dict( range=[ vmin,vmax] ) #Set value limits at both ends
    xaxiser=dict( range=[ xmin,xmax] ) #Set value limits at both ends
    
    if Color_Titler == "":
        Color_Titler= Fact1
    else:

        cmin = min(df[Color_Titler])
        cmax=  max(df[Color_Titler])
    
    if category_order == True: #If you want there to be a certain order to the data.
        if category_order_array=={}:
            if Sort_fact == "":
                     category_order={'categoryorder':'array',
                                     'categoryarray':constant_unique(Fact1,df=df)}
            else:
                    category_order={'categoryorder':'array','categoryarray':constant_unique(Fact1,Sort_fact,df)}
        else:
            category_order=category_order_array 
    else:
        category_order={'categoryorder':'total ascending'}
    #df = df.sort_values(by=[slider_text])
    if slider_text == "":
        fig = px.bar(df, x=df[Fact1], y=df[Value], color=df[Color_Titler], barmode = barmoder , color_discrete_map = constant_color_assigner(df[Fact1]))

    else:
        if Preset_Order == []:
            Preset_Order= list(df[slider_text].sort_values().unique())

        fig = px.bar(df, x=Fact1, y=Value, color=Color_Titler, text=Value,  range_color=[cmin,cmax]
                    , animation_frame=slider_text, barmode = barmoder
                    , color_discrete_map = constant_color_assigner(df[Fact1])
                    ,category_orders={slider_text: Preset_Order})

    fig.update_layout(title=titler   , xaxis_title=Fact1,   xaxis=xaxiser,yaxis= yaxiser)
    fig.update_layout(barmode=barmoder)
    fig.update_layout( margin=dict(l=80, r=80, t=100, b=80),paper_bgcolor="LightSteelBlue",)
    
    fig.show()  
    
def constant_color_assigner(column):
    COLOR_DICT={}
    color_lister = ['yellow','blue','green','red','orange','purple','pink']
    color_lister = color_lister+color_lister
    col_uniques = column.unique()
    col_uniques.sort() #Get constant order for these values

    for val in range(len(col_uniques)): #Iterate through list getting unique index each time
        COLOR_DICT[col_uniques[val]] = color_lister[val]
        if val%7==0:
            color_lister += color_lister

    return COLOR_DICT

In [3]:
#Import Statements
from IPython.core.display import HTML
import seaborn as sns
import csv
import matplotlib.pyplot as plt
import pandas as pd
import dask.dataframe

import pandas as pd
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import random
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeClassifier
import plotly.offline as py
import plotly.graph_objs as go

import plotly.io as pio
pio.renderers.default = "notebook"
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
from tqdm import tqdm
import math
import random

## Get Game Win Probs from FiveThirtyEight

In [4]:
NFL_ELO=pd.read_csv("https://projects.fivethirtyeight.com/nfl-api/nfl_elo_latest.csv")

In [5]:
NFL_ELO=NFL_ELO.sort_values(by='date')

## Find Postseason Projection for each team

In [6]:
url_playoff="http://www.playoffstatus.com/nfl/nflpostseasonprob.html"
#Create a handle, page, to handle the contents of the website
page = requests.get(url_playoff)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [7]:

col=[]
i=0
df=pd.DataFrame()
Column_header=['Team','Conference','W','L','T','Super Bowl Winner','Super Bowl','Conference Championship','Round 2','Round 1']
for t in Column_header:

    col.append((t,[]))
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=len(Column_header):
        pass
        

    #i is the index of our column
    i=0

    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass

        #Append the data to the empty list of the i'th column
        if ((str(col[i][0])) in str(data)):
            pass
        else:
            col[i][1].append(data)

        #Increment i for the next column
        i+=1

## All Teams Playoff Postseason Odds Dataframe

In [8]:
Dict={title:column for (title,column) in col}
All_Playoffs=df.append(pd.DataFrame(Dict)[1:])

## Specifically Colts Chances at Each Round of Playoffs

In [9]:
Colts_Round_1_Chance=list(All_Playoffs[All_Playoffs["Team"]=="Colts"]["Round 1"])[0][:-1]
if Colts_Round_1_Chance=='<1':
    Colts_Round_1_Chance=.001
else:
    Colts_Round_1_Chance=float(Colts_Round_1_Chance)/100
Colts_Round_2_Chance=(list(All_Playoffs[All_Playoffs["Team"]=="Colts"]["Round 2"])[0][:-1])
if Colts_Round_2_Chance=='<1':
    Colts_Round_2_Chance=.001
else:
    Colts_Round_2_Chance=float(Colts_Round_2_Chance)/100
Colts_Conference_Championship_Chance=(list(All_Playoffs[All_Playoffs["Team"]=="Colts"]["Conference Championship"])[0][:-1])
if Colts_Conference_Championship_Chance=='<1':
    Colts_Conference_Championship_Chance=.001
else:
    Colts_Conference_Championship_Chance=float(Colts_Conference_Championship_Chance)/100
Colts_Super_Bowl_Chance=(list(All_Playoffs[All_Playoffs["Team"]=="Colts"]["Super Bowl"])[0][:-1])
if Colts_Super_Bowl_Chance=='<1':
    Colts_Super_Bowl_Chance=.001
else:
    Colts_Super_Bowl_Chance=float(Colts_Super_Bowl_Chance)/100
Colts_Super_Bowl_Winner_Chance=(list(All_Playoffs[All_Playoffs["Team"]=="Colts"]["Super Bowl Winner"])[0][:-1])
if Colts_Super_Bowl_Winner_Chance=='<1':
    Colts_Super_Bowl_Winner_Chance=.001
else:
    Colts_Super_Bowl_Winner_Chance=float(Colts_Super_Bowl_Winner_Chance)/100

## Find Current Carson Wentz Snap Percentage

In [10]:
url="https://www.fantasypros.com/nfl/reports/snap-counts/qb.php?year=2021&show=perc&print=true"

#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [11]:
col=[]
i=0
df=pd.DataFrame()
Column_header=tr_elements[0]
for t in Column_header:
    i+=1
    name=t.text_content()
    #print(name)
    col.append((name,[]))
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]

    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=len(Column_header):
        break

    #i is the index of our column
    i=0

    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass

        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Dict={title:column for (title,column) in col}
df=df.append(pd.DataFrame(Dict)[1:])
Current_Carson_Snap_Percentage=int(list(df[df["Player"]=="Carson Wentz "]["2"])[0][:-1])#[-1:]

## List of Team Abbreviations

In [12]:
Teams_list=list(NFL_ELO.team1.unique())

AFC_TEAMS=['TEN','BUF','CIN','IND','HOU','KC','NE','OAK','MIA','PIT','CLE','JAX','NYJ','LAC','BAL','DEN']


### Playoff adjusted for picks 19-32
Adjusting for playoff odds is a little hard with what 538 gave me. Instead, I'm using "http://www.playoffstatus.com/nfl/nflpostseasonprob.html" which has the chances of Colts winning each theoretical playoff game

In [13]:
Colts_Round2_chance_bayes_fix=Colts_Round_2_Chance/Colts_Round_1_Chance
Colts_Conference_chance_bayes_fix=Colts_Conference_Championship_Chance/Colts_Round_2_Chance
Colts_Super_Bowl_Chance_bayes_fix=Colts_Super_Bowl_Chance/Colts_Conference_Championship_Chance
Colts_Super_Bowl_win_Chance_bayes_fix=Colts_Super_Bowl_Winner_Chance/Colts_Super_Bowl_Chance

## Actual Simulation Loop (Takes up to 15 minutes, sorry)

In [ ]:
Indiana_Only=NFL_ELO[(NFL_ELO["team1"]=="IND")| (NFL_ELO["team2"]=="IND")] # Get only indiana games

# Random number between 0-1
IND_COLTS_DRAFT_PICK=[]


for sim in tqdm(range(Sims)): #100 sims
    Season_Win_Totals=pd.DataFrame(index=Teams_list,columns=["Wins","Losses"]) #
    Season_Win_Totals=Season_Win_Totals.fillna(0)
    Second_Round_Adjuster=0
    Percentage_of_Total_Snaps=0
    Wins_First_Quarter=0
    Wins_Midseason=0
    Wins_Third_Quarter=0
    Ind_Made_Playoffs=False
    Out_of_Division=False
    Out_of_Wildcard=False
    Bench_Forever=False#Whether you've benched Carson Forever at this point
    Game_Where_Indy_Fell_Out_of_Wildcard=None #Fill if Indy Falls out of WIldcard
    Game_Where_Indy_Fell_Out_of_Division=None #Fill if Indy Falls out of Div
    Game_Where_Indy_Fell_Out_of_Playoffs=None
    Wins_of_Div_Winner=0
    Wins_of_Last_WC=0
    for index, row in NFL_ELO.iterrows():
        
        if math.isnan(row["score1"]): # if games hasn't happened yet
            Prob_of_team1_win=int(row["qbelo_prob1"]*1000) #Out of 1000 chance

            Win_Decision=int(random.uniform(0, 1)*1000) #Random number out of 100, do this for .1% chance of tie
            if Prob_of_team1_win > Win_Decision:
                row["score1"] =1
                row["score2"] =0
            elif Prob_of_team1_win == Win_Decision:
                row["score1"] =1
                row["score2"] =1
            else:
                row["score1"] =0
                row["score2"] =1
        if row["score1"] > row["score2"]: #Team 1 Wins
            Season_Win_Totals.at[row["team1"],"Wins"]+=1
            Season_Win_Totals.at[row["team2"],"Losses"]+=1
        elif row["score1"] == row["score2"]: #Team 1 Wins
            Season_Win_Totals.at[row["team1"],"Wins"]+=.5
            Season_Win_Totals.at[row["team2"],"Losses"]+=.5
        else:
            Season_Win_Totals.at[row["team2"],"Wins"]+=1
            Season_Win_Totals.at[row["team1"],"Losses"]+=1
        #Each Part of Season Check
        

        #Carson Injury Counter
        #https://www.draftsharks.com/injury-predictor
        #Check in on Indy
        if row["team1"]=="IND" or row["team2"]=="IND": #Current row is Indy
            
            #When should they start benching him forever:
            #Greater than 5 games under ,500
            Total_Indy_Games_So_Far=Season_Win_Totals.at["IND","Losses"] + Season_Win_Totals.at["IND","Wins"]
            Total_IND_Wins=Season_Win_Totals.at["IND","Wins"]
            Total_TEN_Wins=Season_Win_Totals.at["TEN","Wins"]
            Total_JAX_Wins=Season_Win_Totals.at["JAX","Wins"]
            Total_HOU_Wins=Season_Win_Totals.at["HOU","Wins"]
            Indy_Games_Remaing=17-Total_Indy_Games_So_Far
            #If you're totally out of the division, bench him forever
            if (Indy_Games_Remaing<(Total_TEN_Wins-Total_IND_Wins)) or (Indy_Games_Remaing<(Total_JAX_Wins-Total_IND_Wins)) or(Indy_Games_Remaing<(Total_HOU_Wins-Total_IND_Wins)):
                if Out_of_Division ==False:
                        Game_Where_Indy_Fell_Out_of_Division=Total_Indy_Games_So_Far
                        if Out_of_Wildcard==True and Out_of_Division==False: #Both out of division and wildcard
                            Game_Where_Indy_Fell_Out_of_Playoffs=Total_Indy_Games_So_Far
                Out_of_Division=True #Affect Wildcard Decision
                if random.uniform(0, 1)> .3: #Made up Chance that they don't bench Carson despite lost season

                    Bench_Forever=True
            if (Season_Win_Totals.at["IND","Losses"] - Season_Win_Totals.at["IND","Wins"]) > 5: #Totally out of it, goes to second round
                if random.uniform(0, 1)> .3 and Out_of_Wildcard: #Made up Chance that they don't bench Carson despite lost season
                    Bench_Forever=True
                    
            #Check if out of Wildcard
            Season_Win_Totals_temp = Season_Win_Totals.sample(frac=1)
            AFC_RECORDS_temp=Season_Win_Totals_temp[Season_Win_Totals_temp.index.isin(AFC_TEAMS)]
            AFC_RECORDS_temp = AFC_RECORDS_temp.sample(frac=1)
            AFC_RECORDS_temp=AFC_RECORDS_temp.sort_values(by="Wins").reset_index()
            AFC_RECORDS_temp["AFC SEEDING"]=AFC_RECORDS_temp.index+1
            
            #Check INdy position in AFC
            IND_POS_IN_AFC=AFC_RECORDS_temp[AFC_RECORDS_temp["index"]=="IND"]
            if int(IND_POS_IN_AFC["AFC SEEDING"])<=9:#Currently out of Wildcard 
                Second_Place_WC_Team=AFC_RECORDS_temp[AFC_RECORDS_temp["AFC SEEDING"]==10] #Find 2nd Wildcard Team
                Second_Place_WC_Team_Wins=float(Second_Place_WC_Team["Wins"])
                Second_Place_WC_Team_Name=list(Second_Place_WC_Team["index"])[0]
                if Indy_Games_Remaing<(Second_Place_WC_Team_Wins-Total_IND_Wins): #Can't catch up to Second Place WC
                    if Out_of_Wildcard ==False:
                        Game_Where_Indy_Fell_Out_of_Wildcard=Total_Indy_Games_So_Far
                        if Out_of_Wildcard==False and Out_of_Division==True: #Both out of division and wildcard
                            Game_Where_Indy_Fell_Out_of_Playoffs=Total_Indy_Games_So_Far
                    Out_of_Wildcard=True
                    
                    #print(Second_Place_WC_Team_Name + " has "+str(Second_Place_WC_Team_Wins)+ " wins with "+str(Indy_Games_Remaing)+ " games remaining while Indy has"+ str(Total_IND_Wins))
                    if random.uniform(0, 1)> .3 and Out_of_Division==True: #Made up Chance that they don't bench Carson despite lost season
                        
                        Bench_Forever=True                

            #How much less they'll use Carson
            if (Season_Win_Totals.at["IND","Losses"] - Season_Win_Totals.at["IND","Wins"]) >1: #If they are down 2+ losses, they'll use Carson Less
                Losses_minus_wins=Season_Win_Totals.at["IND","Losses"] - Season_Win_Totals.at["IND","Wins"] #Carson Struggling, bring prob down
            else:
                Losses_minus_wins=1 #Otherwise play normal
                
                
            if (Season_Win_Totals.at["IND","Losses"] + Season_Win_Totals.at["IND","Wins"])==4:
                
                Wins_First_Quarter= int(Season_Win_Totals.at["IND","Wins"])
            if (Season_Win_Totals.at["IND","Losses"] + Season_Win_Totals.at["IND","Wins"])==8:
                Wins_Midseason= int(Season_Win_Totals.at["IND","Wins"])
            if (Season_Win_Totals.at["IND","Losses"] + Season_Win_Totals.at["IND","Wins"])==12:
                Wins_Third_Quarter= int(Season_Win_Totals.at["IND","Wins"])
            try: #If there is info present on his snap percentage
                Current_Carson_Snap_Percentage=int(list(df[df["Player"]=="Carson Wentz "][str(Season_Win_Totals.at["IND","Losses"] + Season_Win_Totals.at["IND","Wins"])])[0][:-1])#[-1:]
                Percentage_of_Total_Snaps+=Current_Carson_Snap_Percentage/100
            except: #If there is no info on his snap percentage
                if Bench_Forever==True: #He's not getting back in the game because the coach decided the season was over
                    Percentage_of_Total_Snaps+=0 #No more Snaps for Carson
                else:
                    if random.uniform(0, 1)<Carson_Injury_Risk: #How much he plays in that game
                        Percentage_of_Total_Snaps+=random.uniform(0, 1)#Misses a random amount of game due do injury
                    else:
                        if Losses_minus_wins ==1: #If he has more wins than losses, or is .500
                            Percentage_of_Total_Snaps+=1
                        else:
                            Percentage_of_Total_Snaps+=.98**(Losses_minus_wins) #Each loss greater than win bring Carsons Playing down

        
    #AFC ONLY
    Season_Win_Totals = Season_Win_Totals.sample(frac=1)
    AFC_RECORDS=Season_Win_Totals[Season_Win_Totals.index.isin(AFC_TEAMS)]
    AFC_RECORDS = AFC_RECORDS.sample(frac=1)
    AFC_RECORDS=AFC_RECORDS.sort_values(by="Wins").reset_index()
    AFC_RECORDS["AFC SEEDING"]=AFC_RECORDS.index+1
    IND_AFC_PLACEMENT=int(list(AFC_RECORDS[AFC_RECORDS["index"]=="IND"]["AFC SEEDING"])[0])
    #TOTAL NFL
    
    #NFC ONLY
    Season_Win_Totals = Season_Win_Totals.sample(frac=1)
    NFC_RECORDS=Season_Win_Totals[~Season_Win_Totals.index.isin(AFC_TEAMS)]
    NFC_RECORDS = NFC_RECORDS.sample(frac=1)
    NFC_RECORDS=NFC_RECORDS.sort_values(by="Wins").reset_index()
    NFC_RECORDS["AFC SEEDING"]=NFC_RECORDS.index+1    
    Season_Win_Totals=Season_Win_Totals.sort_values(by="Wins").reset_index()
    

    

    #Get Draft Pick from overall NFL order
    
    #Get INdys placement in AFC
    
    #Wins for each AFC SOUTH team
    IND_wins=float(Season_Win_Totals[Season_Win_Totals["index"]=="IND"]["Wins"])
    HOU_wins=float(Season_Win_Totals[Season_Win_Totals["index"]=="HOU"]["Wins"])
    JAX_wins=float(Season_Win_Totals[Season_Win_Totals["index"]=="JAX"]["Wins"])
    TEN_wins=float(Season_Win_Totals[Season_Win_Totals["index"]=="TEN"]["Wins"])
    
    #Division Winner Wins
    Wins_of_Div_Winner=max([IND_wins,HOU_wins,JAX_wins,TEN_wins])
    Wins_of_Last_WC=float(AFC_RECORDS[AFC_RECORDS["AFC SEEDING"]==10]["Wins"])
    
    ##Playoff Seeding
    if (IND_AFC_PLACEMENT>9) or ((IND_wins>HOU_wins)and(IND_wins>JAX_wins)and(IND_wins>TEN_wins)): #Makes Playoffs
        Ind_Made_Playoffs= True

        if random.uniform(0, 1)>Colts_Round2_chance_bayes_fix:  #Colts lose in First Round in Random Sim
            Draft_Pick=19 + int(random.uniform(0, 1)*6)
            Playoff_Result = "First Round Exit"
        else:
                if random.uniform(0, 1)>Colts_Conference_chance_bayes_fix:
                    Draft_Pick=25 + int(random.uniform(0, 1)*4)
                    Playoff_Result = "Second Round Exit"
                else:
                    if random.uniform(0, 1)>Colts_Super_Bowl_Chance_bayes_fix:
                        Draft_Pick=29 + int(random.uniform(0, 1)*2) #It's actually a combination of their regular season record
                        Playoff_Result = "Conference Final Exit"
                    else:
                        if random.uniform(0, 1)>Colts_Super_Bowl_win_Chance_bayes_fix:
                            Draft_Pick=31
                            Playoff_Result = "Super Bowl Exit"
                        else:
                            Draft_Pick=32
                            Playoff_Result = "Super Bowl Win"
            
    else:
        Playoff_Result = "Missed Playoffs"
        AFC_PLAYOFF_TEAMS=list(AFC_RECORDS[AFC_RECORDS["AFC SEEDING"]>9]["index"].values)
        NFC_PLAYOFF_TEAMS=list(NFC_RECORDS[NFC_RECORDS["AFC SEEDING"]>9]["index"].values)
        All_Playoff_Teams=AFC_PLAYOFF_TEAMS+NFC_PLAYOFF_TEAMS
        Season_Win_Totals=Season_Win_Totals[~Season_Win_Totals["index"].isin(All_Playoff_Teams)]
        Season_Win_Totals=Season_Win_Totals.sort_values(by="Wins").reset_index()
        Season_Win_Totals["Draft Pick Interpolated"]=Season_Win_Totals.index+1
        Draft_Pick=int(Season_Win_Totals[Season_Win_Totals["index"]=="IND"]["Draft Pick Interpolated"])
    
    if (Percentage_of_Total_Snaps/17)<.7 and Ind_Made_Playoffs== True: #Makes playoffs but <70% snaps
        Second_Round_Adjuster=1 #Carson didn't play enough and they missed the playoffs
    elif (Percentage_of_Total_Snaps/17)<.75:#Percentage of Total Snaps
        Second_Round_Adjuster=1       
    IND_COLTS_DRAFT_PICK.append([Draft_Pick+(32*Second_Round_Adjuster),IND_wins,Wins_First_Quarter,Wins_Midseason,Wins_Third_Quarter,Percentage_of_Total_Snaps
                                 ,Ind_Made_Playoffs,Playoff_Result,Game_Where_Indy_Fell_Out_of_Playoffs,Bench_Forever,Out_of_Division,Game_Where_Indy_Fell_Out_of_Division,Wins_of_Div_Winner,
                                 Out_of_Wildcard,Game_Where_Indy_Fell_Out_of_Wildcard,Wins_of_Last_WC])

 20%|███████▌                              | 1982/10000 [03:04<13:01, 10.26it/s]

## Compile DataFrame of Simulations

In [ ]:
IND_WINS=pd.DataFrame(IND_COLTS_DRAFT_PICK,columns=['Final Pick','Total Projected Indy Wins','First Quarter Wins','Second Quarter Wins','Third Quarter Wins',"Games Played By Carson (17 Max)","Colts Made Playoffs"
                                                    ,"Playoff_Result","Game_Where_Indy_Fell_Out_of_Playoffs","Carson Benched On Purpose","Out_of_Division",
                                                    "Game_Where_Indy_Fell_Out_of_Division","Wins_of_Div_Winner","Out_of_Wildcard",
                                                   "Game_Where_Indy_Fell_Out_of_Wildcard","Wins_of_Last_WC"])

## Average Snap Percentage by Carson

In [ ]:
IND_WINS["Carson Final Snap %"]=IND_WINS["Games Played By Carson (17 Max)"]/17

In [ ]:
print("Carson's Average Final Expected Overall Snap Percentage is {}%".format(str(round(IND_WINS["Carson Final Snap %"].mean()*100,3))))

# Average Draft Pick Colts

In [ ]:
IND_COLTS_DRAFT_PICK=IND_WINS['Final Pick']
print("The Average Compensation Pick From Colts to the Eagles is Pick #{}".format(str(sum(IND_WINS['Final Pick'])/len(IND_WINS['Final Pick']))))

## Highest Occuring Draft Pick

In [ ]:
IND_COLTS_DRAFT_PICK=IND_WINS['Final Pick']
print("The Most Likely Compensation Pick From Colts to the Eagles is Pick #{}".format(str(list(IND_WINS['Final Pick'].mode())[0])))

## Histogram of All Draft Picks

In [ ]:

data = list(IND_WINS['Final Pick'])

# fixed bin size
bins = np.arange(-100, 100, 1) # fixed bin size


plt.xlim(0, 64)

plt.hist(data, bins=bins, alpha=0.5,weights=np.ones(len(data)) / len(data))
plt.title('Carson Wentz Trade Comp. Pick over {} simulations'.format(str(Sims)))
plt.xlabel('Final Projected Compensation Pick'.format(str(Sims)))
plt.ylabel('Percentage of Total Outcomes')

plt.show()

## CDF of Draft Picks

In [ ]:
Eagles_First_Round_DP_list=[]
for j in range(0,66):
    Eagles_First_Round_DP_list.append([j,float(str(sum(i < j for i in IND_COLTS_DRAFT_PICK)/len(IND_COLTS_DRAFT_PICK)*100)[:5])])

In [ ]:
Eagles_First_Round_DP_list_df=pd.DataFrame(Eagles_First_Round_DP_list,columns=['Final Pick','Chance of Pick Higher Than This'])
Plotly_Bar_Plot(df=Eagles_First_Round_DP_list_df,Fact1='Final Pick'
               ,Value='Chance of Pick Higher Than This',titler='CDF of Final Pick Chance')

## Chance of Draft Pick in the First Round

In [ ]:
Eagles_First_Round_DP=str(sum(i < 33 for i in IND_COLTS_DRAFT_PICK)/len(IND_COLTS_DRAFT_PICK)*100)[:5]
print("The Eagles received a 1st Round Pick in {}% of all simulations.".format(Eagles_First_Round_DP))

## Chance of Top 20 Pick

In [ ]:
Eagles_First_Round_DP=str(sum(i < 21 for i in IND_COLTS_DRAFT_PICK)/len(IND_COLTS_DRAFT_PICK)*100)[:5]
print("The Eagles received a pick in the top 20 in {}% of all simulations.".format(Eagles_First_Round_DP))

## Chance of Top 10 Pick

In [ ]:
Eagles_First_Round_DP=str(sum(i < 11 for i in IND_COLTS_DRAFT_PICK)/len(IND_COLTS_DRAFT_PICK)*100)[:5]
print("The Eagles received a pick in the top 10 in {}% of all simulations.".format(Eagles_First_Round_DP))

## Chance of Top 5 Pick

In [ ]:
Eagles_First_Round_DP=str(sum(i < 6 for i in IND_COLTS_DRAFT_PICK)/len(IND_COLTS_DRAFT_PICK)*100)[:5]
print("The Eagles received a pick in the top 5 in {}% of all simulations.".format(Eagles_First_Round_DP))

## Lowest Probable Pick

In [ ]:
max(IND_COLTS_DRAFT_PICK)

## Highest Probable Pick

In [ ]:
min(IND_COLTS_DRAFT_PICK)

## Average Pick By First Quarter Result

In [ ]:
IND_WINS.groupby(['First Quarter Wins'])['Final Pick'].mean()

## Average Pick by Halfway Point Result

In [ ]:
IND_WINS.groupby(['Second Quarter Wins'])['Final Pick'].mean()

## Average Pick by Third Quarter Result

In [ ]:
IND_WINS.groupby(['Third Quarter Wins'])['Final Pick'].mean()

## Desired Wins By Third Quarter for Eagles
### This number of wins by week 13 would represent the lowest average comp. pick

In [ ]:
list(IND_WINS.groupby(['Third Quarter Wins'],as_index=False)['Final Pick'].mean().sort_values(by='Final Pick')['Third Quarter Wins'])[0]

In [ ]:
#Each instance of a final pick is a percenage point
IND_WINS['Percentage of Total Occurances (%)']=round((1/Sims)*100,4)
IND_WINS["Colts Made Playoffs"]=(IND_WINS["Colts Made Playoffs"]).astype(str)

## Draft Pick by Time Out of Playoff Picture

In [ ]:
Wins_by_First_Quarter_Result=IND_WINS.groupby(['Final Pick','Game_Where_Indy_Fell_Out_of_Playoffs'],as_index=False)['Percentage of Total Occurances (%)'].sum()
Carson_Final_Snap_Percentage=IND_WINS.groupby(["Final Pick",'Game_Where_Indy_Fell_Out_of_Playoffs'],as_index=False)['Carson Final Snap %'].mean()
Wins_by_First_Quarter_Result=pd.merge(Wins_by_First_Quarter_Result,Carson_Final_Snap_Percentage,how="inner",
        left_on=['Game_Where_Indy_Fell_Out_of_Playoffs',"Final Pick"], right_on=['Game_Where_Indy_Fell_Out_of_Playoffs',"Final Pick"])
Wins_by_First_Quarter_Result=Wins_by_First_Quarter_Result.round(4)
Plotly_Bar_Plot(df=Wins_by_First_Quarter_Result,Fact1='Final Pick',slider_text='Game_Where_Indy_Fell_Out_of_Playoffs',
                Color_Titler='Carson Final Snap %',Value='Percentage of Total Occurances (%)',titler='Projected Final Comp. Pick from Colts to Eagles by Game_Where_Indy_Fell_Out_of_Playoffs')

## Total Wins For Indy and Average Draft Pick

In [ ]:
Wins_by_First_Quarter_Result=IND_WINS.groupby(['Final Pick','Total Projected Indy Wins'],as_index=False)['Percentage of Total Occurances (%)'].sum()
Carson_Final_Snap_Percentage=IND_WINS.groupby(["Final Pick",'Total Projected Indy Wins'],as_index=False)['Carson Final Snap %'].mean()
Wins_by_First_Quarter_Result=pd.merge(Wins_by_First_Quarter_Result,Carson_Final_Snap_Percentage,how="inner",
        left_on=['Total Projected Indy Wins',"Final Pick"], right_on=['Total Projected Indy Wins',"Final Pick"])
Wins_by_First_Quarter_Result=Wins_by_First_Quarter_Result.round(4)
Plotly_Bar_Plot(df=Wins_by_First_Quarter_Result,Fact1='Final Pick',slider_text='Total Projected Indy Wins',
                Color_Titler='Carson Final Snap %',Value='Percentage of Total Occurances (%)',titler='Projected Final Comp. Pick from Colts to Eagles by Total Projected Indy Wins')

## Making Playoffs vs. NOT

In [ ]:
Wins_by_First_Quarter_Result=IND_WINS.groupby(['Final Pick','Colts Made Playoffs'],as_index=False)['Percentage of Total Occurances (%)'].sum()
Carson_Final_Snap_Percentage=IND_WINS.groupby(["Final Pick",'Colts Made Playoffs'],as_index=False)['Carson Final Snap %'].mean()
Wins_by_First_Quarter_Result=pd.merge(Wins_by_First_Quarter_Result,Carson_Final_Snap_Percentage,how="inner",
        left_on=['Colts Made Playoffs',"Final Pick"], right_on=['Colts Made Playoffs',"Final Pick"])
Wins_by_First_Quarter_Result=Wins_by_First_Quarter_Result.round(4)
Plotly_Bar_Plot(df=Wins_by_First_Quarter_Result,Fact1='Final Pick',slider_text='Colts Made Playoffs',
                Color_Titler='Carson Final Snap %',Value='Percentage of Total Occurances (%)',titler='Projected Final Comp. Pick from Colts to Eagles by Colts Made Playoffs')

## Probability Distribution of Final Pick based off of each Quarter (every 4 games) Result

### First Quarter of Season Done

In [ ]:
Wins_by_First_Quarter_Result=IND_WINS.groupby(['Final Pick','First Quarter Wins'],as_index=False)['Percentage of Total Occurances (%)'].sum()
Carson_Final_Snap_Percentage=IND_WINS.groupby(["Final Pick",'First Quarter Wins'],as_index=False)['Carson Final Snap %'].mean()
Wins_by_First_Quarter_Result=pd.merge(Wins_by_First_Quarter_Result,Carson_Final_Snap_Percentage,how="inner",
        left_on=['First Quarter Wins',"Final Pick"], right_on=['First Quarter Wins',"Final Pick"])
Wins_by_First_Quarter_Result=Wins_by_First_Quarter_Result.round(4)
Plotly_Bar_Plot(df=Wins_by_First_Quarter_Result,Fact1='Final Pick',slider_text='First Quarter Wins',
                Color_Titler='Carson Final Snap %',Value='Percentage of Total Occurances (%)',titler='Projected Final Comp. Pick from Colts to Eagles by Wins at First Quarter')

### Season Halfway Done

In [ ]:
Wins_by_First_Quarter_Result=IND_WINS.groupby(['Final Pick','Second Quarter Wins'],as_index=False)['Percentage of Total Occurances (%)'].sum()

Carson_Final_Snap_Percentage=IND_WINS.groupby(["Final Pick",'Second Quarter Wins'],as_index=False)['Carson Final Snap %'].mean()
Wins_by_First_Quarter_Result=pd.merge(Wins_by_First_Quarter_Result,Carson_Final_Snap_Percentage,how="inner",
        left_on=['Second Quarter Wins',"Final Pick"], right_on=['Second Quarter Wins',"Final Pick"])
Plotly_Bar_Plot(df=Wins_by_First_Quarter_Result,Fact1='Final Pick',slider_text='Second Quarter Wins',
                Color_Titler='Carson Final Snap %',Value='Percentage of Total Occurances (%)',titler='Projected Final Comp. Pick from Colts to Eagles by Wins at Halfway',vmax=2.3)

### Third Quarter Done

In [ ]:
Wins_by_First_Quarter_Result=IND_WINS.groupby(['Final Pick','Third Quarter Wins'],as_index=False)['Percentage of Total Occurances (%)'].sum()

Carson_Final_Snap_Percentage=IND_WINS.groupby(["Final Pick",'Third Quarter Wins'],as_index=False)['Carson Final Snap %'].mean()
Wins_by_First_Quarter_Result=pd.merge(Wins_by_First_Quarter_Result,Carson_Final_Snap_Percentage,how="inner",
        left_on=['Third Quarter Wins',"Final Pick"], right_on=['Third Quarter Wins',"Final Pick"])
Plotly_Bar_Plot(df=Wins_by_First_Quarter_Result,Fact1='Final Pick',slider_text='Third Quarter Wins',
                Color_Titler='Carson Final Snap %',Value='Percentage of Total Occurances (%)',titler='Projected Final Comp. Pick from Colts to Eagles by Wins at 3rd Quarter')

## Analysis

There is no guarantee that the rules of this simulation accurately map out the thought process of the Colts front office. The organization is famously incompetent and prone to misusing their players. Through the first few weeks of this 2021 season, Carson has played admirably under intense pressure from both the media and opposing defensive lines. As long as he is availible and healthy, he will be fighting for his starting job in the league, and the Colts may not be able to bench him so easily even in a losing season if his performance is this good.

Regardless, even after an 1-3 start, this distribution should show Eagles fans that a pick in the top 5 is essentially a pipe dream. 
